In [9]:
import numpy as np
from astropy.io import fits

# Step 1:
* save ejones matrix as real and imaginary numpy arrays

# Step 2:
* make into a complex image
 - To do this, save both np

In [17]:
def arr_2_fits(arr):
    """This function will take a numpy array, save it as a fits file with header information
       and save it as a casa image"""
    
    path = '/lustre/aoc/projects/hera/jaguirre/PolarizedMosaics/2458098_sliced/zen.010601_011009.calibrated.HH.dirty.psf.fits'
    #open arbituary fitsfile to get header information
    data, header = fits.getdata(path, header=True)
    
    #Creates two fitsfiles with the data and header
    real_hdu = fits.PrimaryHDU((arr.real.T),header=header)
    imag_hdu = fits.PrimaryHDU((arr.imag.T),header=header)
    
    #Change header information so it's casa appropiate
    
    #for real
    real_hdu.header['CTYPE4']  = 'FREQ    '                                                            
    real_hdu.header['CRVAL4']  =   1.498214001419E+08                                                  
    real_hdu.header['CDELT4']  =   8.018472876433E+07                                                  
    real_hdu.header['CRPIX4']  =   1.000000000000E+00                                                  
    real_hdu.header['CUNIT4']  = 'Hz      '                                                            
    real_hdu.header['CTYPE3']  = 'STOKES  '                                                            
    real_hdu.header['CRVAL3']  =   1.000000000000E+00                                                  
    real_hdu.header['CDELT3']  =   1.000000000000E+00                                                  
    real_hdu.header['CRPIX3']  =   1.000000000000E+00                                                  
    real_hdu.header['CUNIT3']  = '        '          
    real_hdu.header['RADESYS'] = 'ICRS    '                                                            
    real_hdu.header['CTYPE1']  = 'GLON-TAN'
    real_hdu.header['CTYPE2']  = 'GLAT-TAN'

    #for imaginary
    imag_hdu.header['CTYPE4']  = 'FREQ    '                                                            
    imag_hdu.header['CRVAL4']  =   1.498214001419E+08                                                  
    imag_hdu.header['CDELT4']  =   8.018472876433E+07                                                  
    imag_hdu.header['CRPIX4']  =   1.000000000000E+00                                                  
    imag_hdu.header['CUNIT4']  = 'Hz      '                                                            
    imag_hdu.header['CTYPE3']  = 'STOKES  '                                                            
    imag_hdu.header['CRVAL3']  =   1.000000000000E+00                                                  
    imag_hdu.header['CDELT3']  =   1.000000000000E+00                                                  
    imag_hdu.header['CRPIX3']  =   1.000000000000E+00                                                  
    imag_hdu.header['CUNIT3']  = '        '  
    imag_hdu.header['RADESYS'] = 'GALACTIC    '                                                            
    imag_hdu.header['CTYPE2']  = 'GLAT-TAN'
    imag_hdu.header['CTYPE1']  = 'GLON-TAN'
    
    real = 'real_ejones.fits'
    imag = 'imag_ejones.fits'
    real_hdu.writeto(real,overwrite=True)
    imag_hdu.writeto(imag,overwrite=True)

    return real, imag

# Step 3: Make two fits images into casa images using importfits, and then use ia.makecomplex(after opening the REAL casa image with ia.open()

In [20]:
def making_complex(real,imag):
    """Takes ejones real and imaginary fits files and combines them to make a complex casa image"""
    
    #Create two fits files into casa images
    importfits(fitsimage=real,imagename=real + 'im')
    importfits(fitsimage=imag,imagename=imag + 'im')
    
    #Open the real casa image with casa image analysis, and call ia.makecomplex
    ia.open(real)
    ia.makecomplex('complex.im',imag,overwrite=True)
    compleximage = 'complex.im'
    return  compleximage

# Step 4: Change the coordinate system of the complex image to AZELGEO using casa me tool

In [28]:
def setAZELGEO(lat,lon,ms,nx=512,ny=512):
    """Change the coordinate system into AZELGEO for use with vp.setpbimage and save in a .pb casa image"""
    
    
    a = me.epoch('utc','2017-12-10T18:21:48.369567')
    me.doframe(a) #set time in frame
    me.measure(a,'galactic')
    me.doframe(me.observatory('HERA')) #set position in frame
    b = me.direction('j2000',lat,lon)
    me.measure(b, 'azelgeo') # convert to AZEL
    #open measurement set
    im.open(ms)
    #set the phase center change to AZELGEO
    phcen = me.measure(me.direction('AZELGEO',lat,lon),'AZELGEO')
    
    
    im.defineimage(nx=nx,ny=ny,phasecenter=phcen,stokes='IQUV')            
    
    im.setvp(dovp=True)
    im.makeimage(type='pb', image='herapb.pb')
    im2 = ia.regrid(outfile='myimage', csys=cs.torecord(),
                       overwrite=true)  # Regrid and make new image

    return 

NameError: name 'me' is not defined

# Putting it all together

In [4]:
ejones = np.load('complexjones.npy') #load data
real, imag = arr_2_fits(ejones) #cconvert arrays into fits
compleximage = making_complex(real,imag) #make the complex image


data = np.genfromtxt('/lustre/aoc/projects/hera/gtucker/notebooks/cords.txt') #for phase center
ra0,dec0 = data[0],data[1]
ms = '/lustre/aoc/projects/hera/gtucker/repositories/team_polarization/zen.2458116.24482.xx.HH.uvOCR.ms/'
setAZELGEO(ra0,dec0,ms)

NameError: name 'np' is not defined

In [7]:
import numpy as np
data = np.genfromtxt('/lustre/aoc/projects/hera/gtucker/notebooks/cords.txt') #for phase center
ra0,dec0 = data[0],data[1]
ra0,dec0

(26.74927740904, -30.81088900704)

In [2]:
def AZELGEO_convert(ra,dec):
    a = me.epoch('utc','2017-12-10T18:21:48.369567')
    me.doframe(a) #set time in frame
    me.doframe(me.observatory('HERA')) #set position in frame
    b = me.direction('AZELGEO',ra,dec)
    me.measure(b, ’azel’) # convert to AZEL
    return

In [ ]:
print "\t----\t measure Ex 1 \t----"  
a = me.epoch(’utc’,’today’)          # a time  
print a  
#{’m0’: {’value’: 54054.872957673608, ’unit’: ’d’},  
# ’refer’: ’UTC’,  
# ’type’: ’epoch’}  
print me.doframe(me.source(’1934-638’))  
print me.measure(a, ’tai’) # convert to IAT  
#{’m0’: {’value’: 54054.873339618054, ’unit’: ’d’},  
# ’refer’: ’TAI’,  
# ’type’: ’epoch’}  
print me.doframe(a) # set time in frame  
#True  
print me.doframe(me.observatory(’ALMA’)) # set position in frame  
#True  
b=me.direction(’j2000’, qa.toangle(’0h’), ’-30deg’)  # a direction  
print b  
#{’m0’: {’value’: 0.0, ’unit’: ’rad’},  
# ’m1’: {’value’: -0.52359877559829882, ’unit’: ’rad’},  
# ’refer’: ’J2000’,  
# ’type’: ’direction’}  
print me.measure(b, ’azel’) # convert to AZEL  
#{’m0’: {’value’: 1.9244096810822324, ’unit’: ’rad’},  
# ’m1’: {’value’: 0.76465385681363052, ’unit’: ’rad’},  
# ’refer’: ’AZEL’,  
# ’type’: ’direction’}  
print qa.angle(me.getvalue(me.measure(b,’azel’))[’m0’])     # show as angles  
#[’+110.15.38’]  
print qa.angle(me.getvalue(me.measure(b,’azel’))[’m1’])  
#[’+043.48.41’]  